Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [149]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

Assignment- 
1- Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
2- Engineer at least two new features. (See below for explanation & ideas.)
3- Fit a linear regression model with at least two features.
4- Get the model's coefficients and intercept.
5- Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.
6- What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
**As always, commit your notebook to your fork of the GitHub repo.

In [150]:
# test/train. April+May to train, June to test.
df.dtypes
# the 'created' column seems to hold the only date information and its an object. Hm.

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [151]:
import datetime as dt
df['created'] = df['created'].astype('datetime64[ns]')
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [152]:
df = df.sort_values(by=['created'])

In [160]:
train = df[df['created'] <= '2016-05-31']
test = df[df['created'] >= '2016-06-01']

In [161]:
train

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,feat1,feat2
5186,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,...,0,0,0,0,0,0,0,0,2.0,3
7945,1.0,0,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,...,0,0,0,0,0,0,0,0,1.0,4
6424,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,...,0,0,0,0,0,0,0,1,5.0,9
7719,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,...,0,0,0,0,0,0,0,1,2.0,8
1723,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,...,0,0,0,0,0,0,0,0,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,1.0,3,2016-05-30 16:24:15,THIS IS A STEAL for this fully renovated 3 bed...,IT DOESN'T COST MUCH TO LIVE THE DREAM! Orchar...,40.7219,-73.9880,4400,186 Orchard St,medium,...,0,0,0,0,0,0,0,0,4.0,1
26469,1.0,4,2016-05-30 18:57:50,"Brand new reno, stainless steel appliances inc...",358 Palmetto st,40.6981,-73.9121,3500,358 Palmetto st,low,...,0,0,0,0,0,0,0,1,5.0,6
37746,1.0,0,2016-05-30 20:05:21,"Call, text, or email Mike or Tony to set up a ...",W 15th St and Ave of the Americas,40.7381,-73.9959,3050,77 W 15th St,low,...,0,0,0,0,0,0,0,0,1.0,7
33882,1.0,2,2016-05-30 20:24:17,***NO FEE**\r\rNewly renovated 2 bedroom apart...,Williamsburg,40.7147,-73.9373,2550,955 metropolitan,medium,...,0,0,0,0,0,0,0,0,3.0,3


Engineer 2 new features. Lets do a ratio of bedrooms:bathrooms. And then lets do the total number of 'features' it has (like fast internet, laundry etc)

In [144]:
#df['bedrooms'] = df['bedrooms'].replace(0,1)
#df['bathrooms'] =df['bathrooms'].replace(0,1)
#df

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,feat1,feat2
5186,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,...,0,0,0,0,0,0,0,0,1.0,3
7945,1.0,1,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,...,0,0,0,0,0,0,0,0,1.0,4
6424,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,...,0,0,0,0,0,0,0,1,1.5,9
7719,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,...,0,0,0,0,0,0,0,1,1.0,8
1723,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,...,0,0,0,0,0,0,0,0,1.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19943,1.0,2,2016-06-29 17:47:34,"Newly renovated, clean, quiet & very bright 2...",At Thompson St,40.7261,-74.0014,3045,132 Thompson St #20,medium,...,0,0,0,0,0,0,0,0,2.0,0
16801,1.0,1,2016-06-29 17:56:12,Historic conversion in Greenpoint with awesome...,"100 Dupont St, Brooklyn, NY 11222",40.7358,-73.9560,2648,100 Dupont St,low,...,0,0,0,0,0,0,0,0,1.0,7
32633,1.0,1,2016-06-29 18:14:48,"Large, sunny one bedroom apartment with a sepa...",West 45th St. and 8th Ave.,40.7597,-73.9890,2650,305 West 45th St.,low,...,0,0,0,0,0,0,0,0,1.0,5
20560,1.0,3,2016-06-29 18:30:41,LOOK NO FURTHER!!\r\rMassive three bedroom apa...,williamsburg,40.7156,-73.9540,3200,32 Havemeyer Street,medium,...,0,0,0,0,0,0,0,0,3.0,2


In [154]:
df['feat1'] = df['bedrooms'] + df['bathrooms']

In [155]:
df['feat1']

5186     2.0
7945     1.0
6424     5.0
7719     2.0
1723     2.0
        ... 
19943    3.0
16801    2.0
32633    2.0
20560    4.0
17743    4.0
Name: feat1, Length: 48817, dtype: float64

In [156]:
#feature 2- the sum of all the goodies.
df1 = df.drop(['bathrooms','bedrooms','created','description','display_address',
          'latitude','longitude','price','street_address','interest_level','feat1'], axis=1)

In [167]:
df['feat2'] = df1.sum(axis=1)
print(df.shape)
df.head()

(48817, 36)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,feat1,feat2
5186,1.0,1,2016-04-01 22:12:41,X-LARGE Flex 1BR Loft! ~~ PRIME Greenwich Vill...,Astor Place,40.7302,-73.9924,3195,1 Astor Place,high,...,0,0,0,0,0,0,0,0,2.0,3
7945,1.0,0,2016-04-01 22:56:00,"This Enormous Studio Features: Harwood Floors,...",East 54th Street,40.7576,-73.9677,2000,230 East 54th Street,medium,...,0,0,0,0,0,0,0,0,1.0,4
6424,2.0,3,2016-04-01 22:57:15,--- East 31st St & Lexington Avenue --- This S...,East 31st St & Lexington Avenue,40.7388,-73.9851,5850,105 Lexington Avenue,high,...,0,0,0,0,0,0,0,1,5.0,9
7719,1.0,1,2016-04-01 23:26:07,Reduced Fee!! Priced To Rent!\rLarge Newly Upd...,West End Ave,40.7939,-73.9738,2745,700 West End Ave,medium,...,0,0,0,0,0,0,0,1,2.0,8
1723,1.0,1,2016-04-02 00:48:13,Phenomenal deal of the century!! This spacious...,E 88th street,40.7784,-73.9491,2400,401 E 88th street,medium,...,0,0,0,0,0,0,0,0,2.0,5


In [176]:
#3- Fit a linear regression model with at least two features.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
model = LinearRegression()
feature = ['feat1', 'feat2']
target = ['price']
X_train = train[feature]
X_test = test[feature]
y_train = train[target]
y_test = test[target]

In [177]:
# This is a 0 feature regression.
y_train['price'].mean()

3575.737743019567

In [178]:
model_full = LinearRegression()
model_full.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [183]:
#Mean absolute error for training
y_pred_train = model_full.predict(X_train)
mean_absolute_error(y_train, y_pred_train)

856.6170841813845

In [184]:
#Mean absolute error for testing
y_pred_test = model_full.predict(X_test)
mean_absolute_error(y_test, y_pred_test)

868.0310274660917

In [181]:
#Coefficients
model_full.coef_[0]

array([768.05312122, 101.72758198])

In [186]:
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
print('RMSE for train:', rmse_train)
print('MAE for train:', mae_train)
print('R2 for train:', r2_train)

RMSE for train: 1295.719533595051
MAE for train: 856.6170841813845
R2 for train: 0.4593590832980584


In [188]:
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print('RMSE for train:', rmse_test)
print('MAE for train:', mae_test)
print('R2 for train:', r2_test)

RMSE for train: 1293.2259992338193
MAE for train: 868.0310274660917
R2 for train: 0.4618976702659915
